In [1]:
using Pkg
Pkg.activate("/home/molloi-lab/Desktop/Project BAC/BAC project/libs/")
using Lux, Random, NNlib, Zygote, LuxCUDA, CUDA, FluxMPI, JLD2, DICOM
using Images
using Gtk
using Images
using MLUtils
using Optimisers
using ImageMorphology, ChainRulesCore, Statistics, CSV, DataFrames, Dates, imageToolBox
using XLSX, ImageView
full_data_dir  = "/media/molloi-lab/2TB/BAC/Clean_Dataset_full"
output_data_dir = "/media/molloi-lab/2TB2/BAC ver3/subset2"

  Activating project at `~/Desktop/Project BAC/BAC project/libs`
┌ Warning: MPI Implementation is not CUDA Aware.
└ @ FluxMPI /home/molloi-lab/.julia/packages/FluxMPI/OM5f6/src/FluxMPI.jl:28
Gtk-Message: 20:18:39.433: Failed to load module "canberra-gtk-module"
Gtk-Message: 20:18:39.434: Failed to load module "canberra-gtk-module"


"/media/molloi-lab/2TB2/BAC ver3/subset2"

In [5]:
@load "JLD2s/train_buckets.jld2" train_buckets
@load "JLD2s/test_buckets.jld2" test_buckets

ct = 0 
for i = 1 : 6
    curr = size(unique(train_buckets[i]))[1]
    ct += curr
    println("$i: $curr")
end
println("Train Total = $ct\n")

ct = 0 
for i = 1 : 6
    curr = size(unique(test_buckets[i]))[1]
    ct += curr
    println("$i: $curr")
end
println("Test Total = $ct")

1: 160
2: 32
3: 32
4: 32
5: 23
6: 10
Train Total = 289

1: 40
2: 8
3: 8
4: 8
5: 8
6: 8
Test Total = 80


In [10]:
function img_ct(name, bucket)
    paths = []
    ct = 0
    for i = 1 : 6
        for SID in bucket[i]
            dir1 = joinpath(output_data_dir, name, SID, "cropped")
            for f in readdir(dir1)
                f_name, f_ext = splitext(f)
                if f_ext == ".jld2"
                    push!(paths, joinpath(dir1, f_name))
                    ct += 1
                end
            end
        end
    end
    return paths, ct
end

function distribute_idx(total, ct)
    chunk_size = ceil(Int, total / ct)
    starts = zeros(Int, ct)
    ends = zeros(Int, ct)
    for i = 1 : ct-1
        starts[i] = (i-1)*chunk_size + 1
        ends[i] = i*chunk_size
    end
    starts[end] = total - chunk_size + 1
    ends[end] = total
    return starts ,ends
end

function create_dl(name, bucket)
    paths, ct = img_ct(name, bucket)
    image_container = Array{Any}(undef, ct)
    gt_container = Array{Any}(undef, ct)
    # @Threads.threads for i = 1 : ct
    for i = 1 : ct
        path = paths[i]
        # image
        @load path*".jld2" img_cropped
        x, y = size(img_cropped)
        image_container[i] = reshape(img_cropped, x, y, 1, 1)

        # gt 
        gt = Float32.(Images.load(path*".png"))
        gt_container[i] = round.(reshape(gt, x, y, 1, 1))
    end

    temp_prem = randperm(ct)
    paths = paths[temp_prem]
    image_container = image_container[temp_prem]
    gt_container = gt_container[temp_prem]

    starts, ends = distribute_idx(ct, 4)
    @Threads.threads for j = 1 : 4
        my_data_loader = []
        @save "JLD2s/$(name)_dl_paths_$j.jld2" paths=paths[starts[j] : ends[j]] 
        for i = starts[j] : ends[j]
            push!(my_data_loader, (image_container[i], gt_container[i]))
        end
        @save "JLD2s/$(name)_loader_$j.jld2" data_loader = my_data_loader
    end
end

create_dl (generic function with 1 method)

In [11]:
create_dl("train", train_buckets)

In [12]:
create_dl("test", test_buckets)